## Mongo

In [ ]:
# brew services start mongodb-community@6.0
# on command line

In [ ]:
import pymongo as pm
import pprint as pp

In [ ]:
client = pm.MongoClient()
db = client.ronin
client, db

## Lunar Express

**Lunar Express Items Repository**

https://explorer.roninchain.com/address/ronin:144697847f4bf184534af5945abe0fb5f1b14fba

**Lunar Express Axie Repository**

https://explorer.roninchain.com/address/ronin:27263b825bab7ef905718185342905fbd248d1ed

**Example Lunar Express Burn Transaction**

https://explorer.roninchain.com/tx/0x679d06b10f760cb7eb07f25718278838f53f25be5139ad607555db4fa639212a

**Personal LE Item Receipt**

https://explorer.roninchain.com/tx/0x64ee9f6c4bc4cad1c7cbc5b800f6a22bae916f190e2d63c2ee48c1ac6758d23b

In [ ]:
my_address = '0x16ce38f2f5407fe61d4723a550f14b37650fd074'

addy_LE_items = '0x144697847f4bf184534af5945abe0fb5f1b14fba'
addy_LE_axies = '0x27263b825bab7ef905718185342905fbd248d1ed'
tx_LE_burn = '0x679d06b10f760cb7eb07f25718278838f53f25be5139ad607555db4fa639212a'
tx_LE_item = '0x64ee9f6c4bc4cad1c7cbc5b800f6a22bae916f190e2d63c2ee48c1ac6758d23b'

addy_axie_contract = '0x32950db2a7164ae833121501c797d79e7b79d74c'

In [ ]:
'request to get LE items', db.transactions.count_documents({'to': addy_LE_items})
# 'from' is users

In [ ]:
import pandas as pd

In [ ]:
LE_item_mints = []
cursor = db.transactions.find({'to': addy_LE_items})
for tx in cursor:
    LE_item_mints.append(tx)
LE_item_mints = pd.DataFrame(LE_item_mints)
LE_item_mints

In [ ]:
import datetime as dt

LE_axie_burns = []
cursor = db.axietransfers.find({'to': addy_LE_axies})
for tx in cursor:
    LE_axie_burns.append(tx)
LE_axie_burns = pd.DataFrame(LE_axie_burns)
LE_axie_burns

In [ ]:
db.axietransfers.find_one()

In [ ]:
import plotly.graph_objects as go

In [ ]:
LE_axie_burns['date'] = LE_axie_burns.apply(lambda row : row['created_at'].date(), axis = 1)
LE_axie_burns = LE_axie_burns[LE_axie_burns['date'] <= dt.date(2022, 2, 20)]
LE_axie_burns

### Analysis

Axie transfer data
* get number of axie transfers to `addy_LE_axies`
* general stats
    * how many users participated? absolute count, percentage
    * usage over time
* per user
    * how many uses?
    * usage over time

In [ ]:
usage = LE_axie_burns.groupby(by=['date', 'from']).count()\
    .reset_index()\
    .filter(['date', 'from', 'to'])\
    .rename(columns={'to': 'count'})
usage

In [ ]:
usage_total = usage.groupby(by=['date']).count()\
    .reset_index()
usage_total[:4]

In [ ]:
import datetime as dt

usage_total_during_event = usage_total[usage_total['date'] < dt.date(2022, 2, 21)]
fig = go.Figure(go.Scatter(x=usage_total_during_event['date'], y=usage_total_during_event['count']))
fig.update_layout(
    template='plotly_white', width=400, height=300,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    yaxis_title='number of axies burned', title='Total Usage (Lunar Express)'
)
fig.show()

February 19, 2022 was the last day of the event.

In [ ]:
usage.sort_values(by='count', ascending=False)[:2]

In [ ]:
usage[usage['from']=='0x8197e6b042ddfa7b6d0e8d894b6e58520241a0ef']

In [ ]:
# fig = go.Figure()
# for addy in pd.unique(usage['from']):
#     usage_addy = usage[usage['from']==addy]
#     fig.add_trace(go.Scatter(x=usage_addy['date'], y=usage_addy['count']))
# fig.update_layout(
#     template='plotly_white', width=400, height=300,
#     margin={'l':10, 'r':10, 't':40, 'b':10},
#     yaxis_title='number of axies burned', title='Individual Usage (Lunar Express)'
# )
# fig.show()

In [ ]:
usage_user = usage.groupby(by=['from']).sum()\
    .reset_index()\
    .sort_values(by='count', ascending=False)
usage_user[:4]

In [ ]:
import plotly.express as px

fig = px.histogram(usage_user, x='count', log_y=True, log_x=True)
fig.update_layout(
    template='plotly_white', width=300, height=300,
    margin={'l':10, 'r':10, 't':40, 'b':10},
    yaxis_title='number of axies burned', 
    xaxis_title='number of users',
    title='Usage Distribution by User (Lunar Express)'
)
fig.show()

In [ ]:
import numpy as np

hist, bin_edges = np.histogram(
    usage_user['count'],
    bins=np.logspace(np.log10(1), np.log10(usage_user['count'].max()), 40)
)
fig = go.Figure(go.Scatter(x=bin_edges, y=hist, mode='markers'))
fig.update_layout(
    template='plotly_white', width=400, height=400,
    margin={'l':10, 'r':10, 't':40, 'b':10},
    yaxis={'title': 'number of users', 'type': 'log'},
    xaxis={'title': 'number of axies burned', 'type': 'log'},
    title='Usage Distribution by User (Lunar Express)'
)
fig.show()

hist = hist / sum(hist)
fig = go.Figure(go.Scatter(x=bin_edges, y=hist, mode='markers'))
fig.update_layout(
    template='plotly_white', width=400, height=400,
    margin={'l':10, 'r':10, 't':40, 'b':10},
    yaxis={'title': 'P(number of axies burned)', 'type': 'log'},
    xaxis={'title': 'number of axies burned', 'type': 'log'},
    title='Usage Distribution by User (Lunar Express)'
)
fig.show()

In [ ]:
usage_user_days = usage.groupby(by=['from']).count()\
    .reset_index()\
    .drop('date', axis=1)\
    .rename(columns={'count': 'days'})\
    .sort_values(by='days', ascending=False)
usage_user_days[:3]

In [ ]:
hist, bin_edges = np.histogram(
    usage_user_days['days'],
    bins=np.logspace(np.log10(1), np.log10(usage_user_days['days'].max()), 30)
)
fig = go.Figure(go.Scatter(x=bin_edges, y=hist, mode='markers'))
fig.update_layout(
    template='plotly_white', width=400, height=400,
    margin={'l':10, 'r':10, 't':30, 'b':10},
    yaxis={'title': 'number of users', 'type': 'log'},
    xaxis={'title': 'number of days spent', 'type': 'log'},
    title='Days Spent (Lunar Express)'
)
fig.show()

hist = hist / sum(hist)
fig = go.Figure(go.Scatter(x=bin_edges, y=hist, mode='markers'))
fig.update_layout(
    template='plotly_white', width=400, height=400,
    margin={'l':10, 'r':10, 't':30, 'b':10},
    yaxis={'title': 'P(days spent)', 'type': 'log'},
    xaxis={'title': 'number of days spent', 'type': 'log'},
    title='Days Spent (Lunar Express)'
)
fig.show()

In [ ]:
core_user = usage[usage['from']=='0x9348bfc25963177b9512fbefe89d82dac04dd11f']
core_user['date'].min(), core_user['date'].max(), core_user['date'].max()-core_user['date'].min()

In [ ]:
usage_date_max = usage.groupby(by=['from']).max()\
    .drop('count', axis=1)\
    .rename(columns={'date': 'date_max'})
usage_date_min = usage.groupby(by=['from']).min()\
    .drop('count', axis=1)\
    .rename(columns={'date': 'date_min'})
usage_dates = usage_date_min.join(usage_date_max, on='from')\
    .reset_index()
usage_dates['date_diff'] = usage_dates['date_max'] - usage_dates['date_min'] + dt.timedelta(days=1)
usage_dates['date_diff_days'] = usage_dates['date_diff'].dt.days
usage_dates = usage_dates.sort_values(by='date_diff', ascending=False)
usage_dates

In [ ]:
# average duration participation
import plotly.express as px

fig = px.histogram(usage_dates, x='date_diff_days', log_y=True)
fig.update_layout(
    template='plotly_white', width=400, height=400,
    margin={'l':10, 'r':10, 't':30, 'b':10},
    yaxis_title='number of users', 
    xaxis_title='number of days bt first and last',
    title='Duration of Participation (Lunar Express)'
)
fig.show()

In [ ]:
usage['count'] / usage['count'].sum()

In [ ]:
hist, bin_edges = np.histogram(
    usage['count'],
    bins=np.logspace(np.log10(1), np.log10(usage['count'].max()), 30)
)
fig = go.Figure(go.Scatter(x=bin_edges, y=hist, mode='markers'))
fig.update_layout(
    template='plotly_white', width=400, height=400,
    margin={'l':10, 'r':10, 't':30, 'b':10},
    yaxis={'title': 'number of days', 'type': 'log'},
    xaxis={'title': 'number of burns per day', 'type': 'log'},
    title='Daily Burns (Lunar Express)'
)
fig.show()

hist = hist / sum(hist)
fig = go.Figure(go.Scatter(x=bin_edges, y=hist, mode='markers'))
fig.update_layout(
    template='plotly_white', width=400, height=400,
    margin={'l':10, 'r':10, 't':30, 'b':10},
    yaxis={'title': 'P(daily burns)', 'type': 'log'},
    xaxis={'title': 'number of burns per day', 'type': 'log'},
    title='Daily Burns (Lunar Express)'
)
fig.show()

In [ ]:
fig = px.histogram(usage, x='count', log_y=True, log_x=True)
fig.update_layout(
    template='plotly_white', width=400, height=400,
    margin={'l':10, 'r':10, 't':30, 'b':10},
    yaxis_title='number of days', 
    xaxis_title='number of burns per day',
    title='Daily Burns (Lunar Express)'
)
fig.show()

In [ ]:
# opportunity cost, value burned

In [ ]:
db.axiesales.find_one()

In [ ]:
axie_sales = []
cursor = db.axiesales.find({'created_at': {"$gt": dt.datetime(2022, 1, 19), "$lt": dt.datetime(2022, 2, 21)}})
for tx in cursor:
    axie_sales.append(tx)
axie_sales = pd.DataFrame(axie_sales)
axie_sales

In [ ]:
axie_prices = []
wei_per_eth = 1000000000000000000
window_size = 1000
for i in range(32):
    date_start = dt.datetime(2022, 1, 20) + dt.timedelta(days=i)
    date_end = dt.datetime(2022, 1, 20) + dt.timedelta(days=i+1)
    cursor = db.axiesales.find({
        'created_at': {'$gt': date_start, '$lt': date_end},
        'price': {'$ne': '0'}
    })\
        .sort('price', pm.ASCENDING)
    price_eth = 0
    for i in range(window_size):
        row = next(cursor)
        price_eth += int(row['price']) / wei_per_eth
    price_eth /= window_size
    axie_prices.append(price_eth)
axie_prices

In [ ]:
dates = [dt.datetime(2022, 1, 20) + dt.timedelta(days=i) for i in range(31)]
fig = go.Figure(go.Scatter(x=dates, y=axie_prices))
fig.update_layout(
    template='plotly_white', width=400, height=300,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    yaxis={'title': 'ETH', 'range': [0, max(axie_prices)]},
    title='Axie Prices'
)
fig.show()

- What was the depth of the axie market? How would a burn mechanism affect the market?
- Did people over-react to the event by breeding too much?

In [ ]:
import requests
import time

eth_prices = []
for i in range(32):
    date = dt.datetime(2022, 1, 20) + dt.timedelta(days=i)
    url = "https://api.coingecko.com/api/v3/coins/ethereum/history?date=" + date.strftime("%d-%m-%Y")
    r = requests.get(url)
    eth_usd = r.json()['market_data']['current_price']['usd']
    eth_prices += [eth_usd]
    time.sleep(2)

In [ ]:
usage_total['eth_sunk'] = usage_total['count'] * axie_prices
usage_total['usd_sunk'] = usage_total['eth_sunk'] * eth_prices
usage_total

In [ ]:
import datetime as dt
from plotly.subplots import make_subplots

fig = go.Figure(go.Scatter(x=usage_total['date'], y=usage_total['count']))
fig.update_layout(
    template='plotly_white', width=400, height=300,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    yaxis={'title': 'number of axies burned', 'type': 'log'},
    title='Total Usage (Lunar Express)'
)
fig.show()

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=usage_total['date'], y=usage_total['eth_sunk'], name='ETH'))
fig.add_trace(go.Scatter(x=usage_total['date'], y=usage_total['usd_sunk'], name='USD'), secondary_y=True)
go.Scatter(x=usage_total['date'], y=usage_total['usd_sunk'])
fig.update_layout(
    template='plotly_white', width=480, height=300,
    margin={'l':10, 'r':10, 't':60, 'b':10},
    yaxis={'title': 'value burned', 'type': 'log'},
    title='Total Usage (Lunar Express)'
)
fig.update_yaxes(type='log', secondary_y=True)
fig.show()

In [ ]:
total_eth_sunk = usage_total['eth_sunk'].sum()
total_usd_sunk = usage_total['usd_sunk'].sum()
print(f"Ξ{total_eth_sunk:,.2f}")
print(f"${total_usd_sunk:,.2f}")

In [ ]:
# volume of items traded

In [ ]:
# social metrics

### Conclusions

Most burns occurred at the very beginning. The end of the event

Scale-free... Heavy tails everywhere...